Get the graph linked by the in-links in file resources/wt2g_inlinks.txt.zip
Compute the PageRank of every page. List the top 500 pages by the PageRank score; also display inlink and outlink counts for each page
Explain in few sentences why some pages have a higher PageRank but a smaller inlink count. In particular for finding the explanation: pick such case pages and look at other pages that point to them.

In [19]:
import re
import os
import time
import numpy as np
import scipy.stats
import copy
import random
import math


In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [20]:
class GatherData:
    def __init__(self):
        pass
    
    def read_data(self):
        ## read from hdd
        with open("wt2g_inlinks.txt") as file:
            data = file.readlines()
        
        self.extended = [i.split(" ") for i in data]
    
    def create_inlink(self):
        ## create outlink dictionary
        self.inlink_dict = {}
        for i in self.extended:
            if len(i) > 1:
                if i[-1] == "\n":
                     self.inlink_dict[i[0]] = i[1:-1]
                elif i[-1][-1] == "\n":
                     self.inlink_dict[i[0]] = [i[-1][:-1]]
    
    def create_outlink(self):
        ## create outlink dictionary
        self.outlink_dict = {}
        for key in self.inlink_dict:
            for value in self.inlink_dict[key]:
                if value not in self.outlink_dict:
                    self.outlink_dict[value] = [key]
                else:
                    self.outlink_dict[value].append(key)
    
    def get_data_info(self):
        ## get total num of pages
        self.total_pages = (list(set(list(self.outlink_dict.keys()) + list(self.inlink_dict.keys()))))
        ## initialize value of page rank
        self.num_of_pages = len(self.total_pages)

        self.page_rank = {i: (1/self.num_of_pages) for i in self.total_pages}
        self.pages_with_no_outlink = [i for i in self.total_pages if i not in self.outlink_dict]
        
    

In [21]:

class PageRank(GatherData):
    
    def __init__(self):
        pass
    
    def call_needed_functions(self):
        self.read_data()
        self.create_inlink()
        self.create_outlink()
        self.get_data_info()
        
    
    def calculate_pagerank(self):
        counter = 1
        d = 0.85
        flag = False
        i = 0

        while (True):  
            i += 1
            new_page_rank = {}

            oldentropy = scipy.stats.entropy(list(self.page_rank.values()))
            perplexity_old = 2 ** (oldentropy)

            sink_pr = 0
            for each_page in self.pages_with_no_outlink:
                sink_pr += self.page_rank[each_page]

            for each_page in self.total_pages:
                new_page_rank[each_page] = (1 - d) / self.num_of_pages
                new_page_rank[each_page] += (d * sink_pr / self.num_of_pages)
                if each_page in self.inlink_dict:
                    for page_inlink in self.inlink_dict[each_page]:
                        length_outlink = len(self.outlink_dict[page_inlink])
                        new_page_rank[each_page] += (d * self.page_rank[page_inlink])/length_outlink

            new_entropy = scipy.stats.entropy(list(new_page_rank.values()))
            perplexity_new = 2 ** (new_entropy)

            if np.isclose(perplexity_new, perplexity_old, atol = 0.9):
                counter += 1
                flag = True
            else:
                counter = 1
                flag = False

            if flag == True:
                if counter == 4:
                    break

            print ("At iteration:",i,"perplexity_new: ",perplexity_new, " /***/ perplexity_old: ", perplexity_old )

            self.page_rank = copy.deepcopy(new_page_rank)

        sorted_data = sorted(self.page_rank.items(),key = lambda x:x[1], reverse=True)  
        return sorted_data
    
    
    

In [229]:
page_rank_object = PageRank()
page_rank_object.call_needed_functions()
sorted_data = page_rank_object.calculate_pagerank()

At iteration: 1 perplexity_new:  2496.4242545378575  /***/ perplexity_old:  4456.400980452243
At iteration: 2 perplexity_new:  2637.9659563887076  /***/ perplexity_old:  2496.4242545378575
At iteration: 3 perplexity_new:  2333.0975240063844  /***/ perplexity_old:  2637.9659563887076
At iteration: 4 perplexity_new:  2396.9884989385782  /***/ perplexity_old:  2333.0975240063844
At iteration: 5 perplexity_new:  2258.0857174598714  /***/ perplexity_old:  2396.9884989385782
At iteration: 6 perplexity_new:  2309.2644291852203  /***/ perplexity_old:  2258.0857174598714
At iteration: 7 perplexity_new:  2231.9049697783935  /***/ perplexity_old:  2309.2644291852203
At iteration: 8 perplexity_new:  2268.224665526406  /***/ perplexity_old:  2231.9049697783935
At iteration: 9 perplexity_new:  2222.7940065776165  /***/ perplexity_old:  2268.224665526406
At iteration: 10 perplexity_new:  2247.748508037083  /***/ perplexity_old:  2222.7940065776165
At iteration: 11 perplexity_new:  2218.756202816342  

In [231]:
for i in sorted_data:
    print (i)

('WT21-B37-76', 0.0026800189805408973)
('WT21-B37-75', 0.001526253832298064)
('WT25-B39-116', 0.0014688899524919511)
('WT23-B21-53', 0.0013715986864474457)
('WT24-B40-171', 0.0012470582775339272)
('WT23-B39-340', 0.0012431993456786816)
('WT23-B37-134', 0.0012047296239651316)
('WT08-B18-400', 0.0011436558787307194)
('WT13-B06-284', 0.0011250543204779733)
('WT24-B26-46', 0.001084938130617919)
('WT13-B06-273', 0.001044952007761896)
('WT01-B18-225', 0.0009885519467228432)
('WT04-B27-720', 0.0009369174843650065)
('WT23-B19-156', 0.000894247605660858)
('WT04-B30-12', 0.0008164211548750097)
('WT24-B26-10', 0.0008054732102872363)
('WT25-B15-307', 0.0008044582055442256)
('WT07-B18-256', 0.0007766642689198737)
('WT24-B26-2', 0.0007714444623892792)
('WT14-B03-220', 0.0007180347641086974)
('WT24-B40-167', 0.000708752418447946)
('WT14-B03-227', 0.0006865725106598761)
('WT18-B31-240', 0.0006602783218150452)
('WT04-B40-202', 0.0006590653224767391)
('WT08-B19-222', 0.0006434902794962647)
('WT27-B28-20

In [24]:
class PageRankCrawler(PageRank):
    
    def __init__(self):
        pass

    def crawler_data(self): 
        body = """
        {
          "_source" : ["in_links", "out_links"],
          "query": {
            "match_all":{}
          }
        }
        """
        complete_data = es.search(index = "merge", doc_type= "document")
        complete_data_length = complete_data["hits"]["total"]
        data = es.search(index = "merge", doc_type= "document", body = body, size = 1000)
        total_num_docs = data["hits"]["total"]
        self.inlink_dict = {}
        self.outlink_dict = {}
        total_num_docs = 1000
        for i in range(total_num_docs):
            dictionary_value = data["hits"]["hits"][i]
            id_value = dictionary_value["_id"]
            self.inlink_dict[id_value] = dictionary_value["_source"]["in_links"]
        self.create_outlink()
            
    def call_additional_data_structure_page_rank(self, inlink_dict):
        self.inlink_dict = inlink_dict
        self.create_outlink()
#         self.crawler_data()
        self.get_data_info()
        sorted_crawler_data  = self.calculate_pagerank()
        return (sorted_crawler_data)


In [26]:
object_pagerank_crawler = PageRankCrawler()
sorted_crawler_data = object_pagerank_crawler.call_additional_data_structure_page_rank(inlink_dict)


At iteration: 1 perplexity_new:  1487.9662342248307  /***/ perplexity_old:  2060.1908112435235
At iteration: 2 perplexity_new:  1275.7103492742003  /***/ perplexity_old:  1487.9662342248307
At iteration: 3 perplexity_new:  1146.6813504514016  /***/ perplexity_old:  1275.7103492742003
At iteration: 4 perplexity_new:  1080.460389585189  /***/ perplexity_old:  1146.6813504514016
At iteration: 5 perplexity_new:  1036.5100295409213  /***/ perplexity_old:  1080.460389585189
At iteration: 6 perplexity_new:  1007.9379890725891  /***/ perplexity_old:  1036.5100295409213
At iteration: 7 perplexity_new:  985.5035044317724  /***/ perplexity_old:  1007.9379890725891
At iteration: 8 perplexity_new:  968.8304183946188  /***/ perplexity_old:  985.5035044317724
At iteration: 9 perplexity_new:  954.9571844427542  /***/ perplexity_old:  968.8304183946188
At iteration: 10 perplexity_new:  944.1255492238224  /***/ perplexity_old:  954.9571844427542
At iteration: 11 perplexity_new:  935.0250861063078  /***/

In [28]:
print(sorted_crawler_data[:46])

[('http://en.wikipedia.org/wiki/Category:Hidden_categories', 0.032804599113742255), ('https://www.enable-javascript.com/', 0.017580166455500996), ('https://theguardian.newspapers.com/', 0.007005781048483138), ('http://shipwrecks.slc.ca.gov/Info/Shipwrecks.html', 0.006777193702288915), ('http://shipwrecks.slc.ca.gov/Info/Shipwrecks/ShipwreckInfo.pdf', 0.004023860575241998), ('http://shop.telegraph.co.uk/?utm_campaign=portal&utm_medium=footer&utm_source=tmg', 0.001998837513655814), ('http://shipwrecks.slc.ca.gov/Info/Shipwrecks/Shipwrecks/ShipwreckInfo.pdf', 0.001596041927959417), ('http://en.wikipedia.org/wiki/International_Standard_Book_Number', 0.0014529700012346158), ('https://www.cruiseindustrynews.com/store/product/digital-reports/global-cruise-ship-index/', 0.0012019234503778604), ('http://www.itv.com/presscentre', 0.0011921129769307507), ('https://www.theguardian.com/index/subjects/a', 0.001063344047612598), ('https://www.theguardian.com/help/terms-of-service', 0.0010633440476125

In [443]:
# # Initialize the scroll
# page = es.search(
#   index = 'yourIndex',
#   doc_type = 'yourType',
#   scroll = '2m',
#   search_type = 'scan',
#   size = 1000,
#   body = {
#     # Your query's body
#     })
#   sid = page['_scroll_id']
#   scroll_size = page['hits']['total']
  
#   # Start scrolling
#   while (scroll_size > 0):
#     print "Scrolling..."
#     page = es.scroll(scroll_id = sid, scroll = '2m')
#     # Update the scroll ID
#     sid = page['_scroll_id']
#     # Get the number of results that we returned in the last scroll
#     scroll_size = len(page['hits']['hits'])
#     print "scroll size: " + str(scroll_size)
#     # Do something with the obtained page
    
    
    
complete_data = es.search(index = "merge", doc_type= "document")
complete_data_length = complete_data["hits"]["total"]
data = es.search(index = "merge", doc_type= "document", body = body, size = 2000, scroll = "5m")
for i in range(1,75000):
    body = """
            {
              "_source" : ["in_links", "out_links"],
              "query": {
              "match" : {
                "docno": "%s"
                  }
              }
            }
            """%i
    try:
        data = es.search(index = "merge", doc_type= "document", body = body)
        dictionary_value = data["hits"]["hits"][0]
        id_value = dictionary_value["_id"] 
        inlink_dict[id_value] = dictionary_value["_source"]["in_links"]
        outlink_dict[id_value] = dictionary_value["_source"]["out_links"]
    except:
        print (i)
        continue

In [10]:
body = """
            {
              "_source" : ["in_links", "out_links"],
              "query": {
              "match" : {
                "docno": "%s"
                  }
              }
            }
            """%(1)
data = es.search(index = "merge", doc_type= "document", body = body)
print (data)

{'took': 6, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'failed': 0}, 'hits': {'total': 1, 'max_score': 1.0, 'hits': [{'_index': 'merge', '_type': 'document', '_id': 'http://en.wikipedia.org/wiki/War', '_score': 1.0, '_source': {'out_links': ['http://en.wikipedia.org/wiki/War', 'http://en.wikipedia.org/wiki/War_(disambiguation)', 'http://en.wikipedia.org/wiki/The_War_(disambiguation)', 'http://en.wikipedia.org/wiki/Conflict_Zone', 'http://en.wikipedia.org/wiki/Military_history', 'http://en.wikipedia.org/wiki/Prehistoric_warfare', 'http://en.wikipedia.org/wiki/Ancient_warfare', 'http://en.wikipedia.org/wiki/Medieval_warfare', 'http://en.wikipedia.org/wiki/Early_modern_warfare', 'http://en.wikipedia.org/wiki/Modern_warfare', 'http://en.wikipedia.org/wiki/Industrial_warfare', 'http://en.wikipedia.org/wiki/Fourth-generation_warfare', 'http://en.wikipedia.org/wiki/Battlespace', 'http://en.wikipedia.org/wiki/Aerial_warfare', 'http://en.wikipedia.org/wiki/Cyberwarfare', 'http

In [451]:
len(inlink_dict)

43000

In [437]:
data["hits"]["hits"][0]

{'_id': 'http://en.wikipedia.org/wiki/Lucretius',
 '_index': 'merge',
 '_score': 1.0,
 '_source': {'in_links': ['http://en.wikipedia.org/wiki/Jeremy_Bentham',
   'http://en.wikipedia.org/wiki/Milesian_school',
   'http://en.wikipedia.org/wiki/Latin_language',
   'http://en.wikipedia.org/wiki/Heraclitus',
   'http://en.wikipedia.org/wiki/Roman_mythology',
   'http://en.wikipedia.org/wiki/Julian_calendar',
   'http://en.wikipedia.org/wiki/Ancient_Rome',
   'http://en.wikipedia.org/wiki/Roman_Republic',
   'http://en.wikipedia.org/wiki/Pluralist_school',
   'http://en.wikipedia.org/wiki/Hard_incompatibilism',
   'http://en.wikipedia.org/wiki/Philosophical_skepticism',
   'http://en.wikipedia.org/wiki/Western_Roman_Empire',
   'http://en.wikipedia.org/wiki/History_of_evolutionary_thought',
   'http://en.wikipedia.org/wiki/Plotinus',
   'http://en.wikipedia.org/wiki/Antioch',
   'http://en.wikipedia.org/wiki/Renaissance_humanism',
   'http://en.wikipedia.org/wiki/Fall_of_the_Western_Roman_E

In [420]:
data["_scroll_id"]


'DnF1ZXJ5VGhlbkZldGNoAwAAAAAAAB77FldWLUJrbldoU3FXd0hZMGVNYUk1R0EAAAAAAAAe_BZXVi1Ca25XaFNxV3dIWTBlTWFJNUdBAAAAAAAAHv0WV1YtQmtuV2hTcVd3SFkwZU1hSTVHQQ=='

In [ ]:
complete_data = es.search(index = "merge", doc_type= "document")
complete_data_length = complete_data["hits"]["total"]
data = es.search(index = "merge", doc_type= "document", body = body, size = 1000)

In [317]:
page_rank_crawler_object = PageRankCrawler()
sorted_crawler_data = page_rank_crawler_object.call_additional_data_structure_page_rank()

At iteration: 1 perplexity_new:  84.28908481462531  /***/ perplexity_old:  840.4669709873289
At iteration: 2 perplexity_new:  80.59665603399708  /***/ perplexity_old:  84.28908481462531
At iteration: 3 perplexity_new:  70.7272480511126  /***/ perplexity_old:  80.59665603399708
At iteration: 4 perplexity_new:  67.07966828093716  /***/ perplexity_old:  70.7272480511126
At iteration: 5 perplexity_new:  65.00338336931136  /***/ perplexity_old:  67.07966828093716
At iteration: 6 perplexity_new:  63.79660953292114  /***/ perplexity_old:  65.00338336931136
At iteration: 7 perplexity_new:  63.032619200400475  /***/ perplexity_old:  63.79660953292114
At iteration: 8 perplexity_new:  62.517828537525666  /***/ perplexity_old:  63.032619200400475


In [63]:
with open("sorted_crawler_data.txt", "w") as file:
    file.write(str(sorted_crawler_data))


In [314]:
len(sorted_crawler_data)

100

In [30]:
body = """
        {
          "query": {
            "match":{
            "text":"maritime accidents"
            }
          }
        }
        """
total_num_docs = 1000
        
data = es.search(index = "merge", doc_type= "document", body = body, size = total_num_docs)

In [55]:
root_set = []
for i in range(total_num_docs):
    root_set.append(data["hits"]["hits"][i]["_id"])

base_set = root_set[:]
d_constant = 200
# for index,id_value in enumerate(root_set):
#     dictionary_value = data["hits"]["hits"][index]
#     inlink_dict[id_value] = dictionary_value["_source"]["in_links"]
#     outlink_dict[id_value] = dictionary_value["_source"]["out_links"]
flag = False
for index,id_value in enumerate(root_set):
    base_set.extend(outlink_dict[id_value])
    print (len(set(base_set)))   
    if len(inlink_dict[id_value]) <= d_constant:
        base_set.extend(inlink_dict[id_value])
    else:
        random_indices = random.choices(inlink_dict[id_value], k = d_constant)
        base_set.extend(random_indices)
    if len(set(base_set)) > 10000:
        flag = True
        print ("flag",len(set(base_set)))
        break
base_set = list(set(base_set))

1185
1298
1353
1397
1492
1496
1498
1577
1681
1683
1734
1801
1848
1879
1891
1908
1960
1960
2008
2053
2054
2067
2169
2269
2360
2361
2370
2400
2423
2486
2565
2650
2691
2739
2760
2761
2811
2812
2813
2825
2844
2918
2956
2998
3034
3082
3085
3122
3140
3149
3150
3188
3193
3217
3221
3222
3239
3239
3249
3311
3319
3377
3394
3606
3629
3763
3806
3806
3807
3843
3877
3898
3943
3983
3985
3992
3996
4007
4009
4017
4025
4025
4036
4036
4046
4055
4056
4065
4107
4125
4134
4250
4270
4270
4279
4287
4630
4691
4762
4815
4815
4818
4945
4957
5108
5108
5118
5118
5122
5142
5150
5208
5235
5368
5497
5497
5497
5499
5538
5592
5848
5866
5896
5908
5922
5976
6005
6021
6033
6053
6083
6086
6094
6095
6103
6206
6206
6227
6228
6235
6237
6238
6253
6284
6567
6589
6659
6756
6777
6879
6894
6895
6947
6949
6961
7022
7022
7122
7136
7226
7235
7485
7495
7528
7557
7603
7667
7697
7698
7911
7918
7927
7946
7949
8132
8191
8207
8215
8223
8230
8284
8310
8330
8504
8518
8527
8549
8552
8595
8639
8705
8719
8725
8891
8898
8901
8926
9000
9021
9022


In [54]:
len(base_set)


33697

In [355]:
#  1  G := set of pages
#  2 for each page p in G do
#  3   p.auth = 1 // p.auth is the authority score of the page p
#  4   p.hub = 1 // p.hub is the hub score of the page p
#  5 function HubsAndAuthorities(G)
#  6   for step from 1 to k do // run the algorithm for k steps
#  7     norm = 0
#  8     for each page p in G do  // update all authority values first
#  9       p.auth = 0
# 10       for each page q in p.incomingNeighbors do // p.incomingNeighbors is the set of pages that link to p
# 11          p.auth += q.hub
# 12       norm += square(p.auth) // calculate the sum of the squared auth values to normalise
# 13     norm = sqrt(norm)
# 14     for each page p in G do  // update the auth scores 
# 15       p.auth = p.auth / norm  // normalise the auth values
# 16     norm = 0
# 17     for each page p in G do  // then update all hub values
# 18       p.hub = 0
# 19       for each page r in p.outgoingNeighbors do // p.outgoingNeighbors is the set of pages that p links to
# 20         p.hub += r.auth
# 21       norm += square(p.hub) // calculate the sum of the squared hub values to normalise
# 22     norm = sqrt(norm)
# 23     for each page p in G do  // then update all hub values
# 24       p.hub = p.hub / norm   // normalise the hub values



10019

In [59]:
authority_score = {}
hub_score = {}
for each_page in base_set:
    authority_score[each_page] = 1
    hub_score[each_page] = 1

def calculate_hits():
    i = 1
    while(True):
        norm = 0
        initial_authority_score = copy.copy(authority_score)
        for each_page in root_set:
            authority_score[each_page] = 0

            for inlink_page in inlink_dict[each_page]:
                if inlink_page in hub_score:
                    authority_score[each_page] += hub_score[inlink_page]
            norm += (authority_score[each_page] ** 2)
        norm = math.sqrt(norm)
        for each_page in base_set:
            authority_score[each_page] = authority_score[each_page] / norm
        norm = 0
        initial_hub_score = copy.copy(hub_score)
        for each_page in base_set:
            hub_score[each_page] = 0

            if each_page in outlink_dict:
                for outlink_page in outlink_dict[each_page]:
                    if outlink_page in authority_score:
                        hub_score[each_page] += authority_score[outlink_page]
                norm += (hub_score[each_page] ** 2)
        norm = math.sqrt(norm)
        for each_page in base_set:
            hub_score[each_page] = hub_score[each_page] / norm
        print ("Iteration: ", i)
        i += 1
#         print (list(authority_score.items())[:10], "\n",list(hub_score.items())[:10], "\n")
        if np.allclose(list(initial_authority_score.values()), list(authority_score.values())) and np.allclose(list(initial_hub_score.values()), list(hub_score.values())):
            sorted_authority = sorted(authority_score.items(), key = lambda x: x[1], reverse = True)[:500]
            sorted_hub = sorted(hub_score.items(), key = lambda x: x[1], reverse = True)[:500]
            return (sorted_authority, sorted_hub)
            
(sorted_authority, sorted_hub) = calculate_hits()

Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10


In [60]:
sorted_authority

[('http://disasteratsea.com/', 0.11877263298698208),
 ('http://disasteratsea.com/index.php/category/press-release/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/coast-guard/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/maritime-executive/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/salvage/', 0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/carnival-cruises/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/us-senate/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/anniversary/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/whale/', 0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/resignatioin/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/human-error/',
  0.11765198609555703),
 ('http://disasteratsea.com/index.php/category/editorial/',
  0.11765198609555703),
 ('h

In [61]:
sorted_hub

[('http://disasteratsea.com/index.php/costa-concordia-owner-faces-2-billion-in-costs/',
  0.0815235681094911),
 ('http://disasteratsea.com/index.php/insurers-1-2bn-bill-for-costa-concordia-salvage-contractors-set-to-refloat-stricken-cruise-ship-in-next-two-weeks/',
  0.08150997178133092),
 ('http://disasteratsea.com/index.php/costa-concordia-refloat-date-set/',
  0.08148413327046498),
 ('http://disasteratsea.com/index.php/more-on-the-bloodbath-aboard-cruise-ship/',
  0.08144337036112004),
 ('http://disasteratsea.com/index.php/cruise-ship-returning-to-u-s-port-after-hitting-30-foot-waves/',
  0.08144337036112004),
 ('http://disasteratsea.com/index.php/brawl-cuts-short-legend-cruise/',
  0.08144337036112004),
 ('http://disasteratsea.com/index.php/salvage-company-employees-jailed-for-stealing-costa-concordia-backpack/',
  0.08144337036112004),
 ('http://disasteratsea.com/index.php/dirty-2017-filthy-cruise-year/',
  0.08144337036112004),
 ('http://disasteratsea.com/', 0.08144337036112002),

In [ ]:

# Compute Hubs and Authority score for the pages in the crawl (merged team index)
# A. Create a root set: Obtain the root set of about 1000 documents by ranking all pages using an 
# IR function (e.g. BM25, ES Search). You will need to use your topic as your query

# B. Repeat few two or three time this expansion to get a base set of about 10,000 pages: 
# For each page in the set, add all pages that the page points to
# For each page in the set, obtain a set of pages that pointing to the page
# if the size of the set is less than or equal to d, add all pages in the set to the root set
# if the size of the set is greater than d, add an RANDOM (must be random) set of d pages from the set to the root set
# Note: The constant d can be 200. The idea of it is trying to include more possibly strong hubs into the root set 
# while constraining the size of the root size.
# C. Compute HITS. For each web page, initialize its authority and hub scores to 1. Update hub and authority scores for each page in the base set until convergence

# Authority Score Update: Set each web page's authority score in the root set to the sum of the hub 
#score of each web page that points to it
# Hub Score Update: Set each web pages's hub score in the base set to the sum of the authority score
#of each web page that it is pointing to
# After every iteration, it is necessary to normalize the hub and authority scores. 
#Please see the lecture note for detail.
# Create one file for top 500 hub webpages, and one file for top 500 authority webpages. 
#The format for both files should be:
# [webpageurl][tab][hub/authority score]\n

In [406]:
def write_in_file(filename, sorted_data):
    with open(filename, "w") as file:
        for i in sorted_data:
            str_sorted_data = i[0] + "    " + str(i[1]) + "\n"
            file.writelines(str_sorted_data)

In [407]:
write_in_file("Sorted_hub.txt", sorted_hub)
write_in_file("Sorted_authority.txt", sorted_authority)